**Import Python module**

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
import os
import zipfile
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models
from torch.autograd import Variable

In [ ]:
#Load data to Google Colab
local_zip = '/gdrive/MyDrive/DLH_Project_Data/DLH_Project_Data/archive.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()
DATA_PATH = "Data"

**Data pre-processing(Augmentation + DataLoader)**

In [ ]:
def get_count_metrics(folder, data_path=DATA_PATH):

    train_dir = os.path.join(data_path, folder)
    list_p = os.listdir(os.path.join(train_dir,'PNEUMONIA')) # dir is your directory path
    num_p = len(list_p)
    list_n = os.listdir(os.path.join(train_dir,'NORMAL')) # dir is your directory path
    num_n = len(list_n)
    list_c = os.listdir(os.path.join(train_dir,'COVID19')) # dir is your directory path
    num_c = len(list_c)
    count_tuple = (int(num_n), int(num_p), int(num_c))

    return count_tuple

def load_data(data_path=DATA_PATH):
    
    transform_dict = {
        'train': transforms.Compose(
        [transforms.Resize(224),
         transforms.RandomResizedCrop(224),
         transforms.RandomHorizontalFlip(p=0.5),
         transforms.RandomRotation(degrees=(-10, 10)),
         transforms.RandomVerticalFlip(p=0.5),
         transforms.GaussianBlur(kernel_size= (5,5),sigma=(0.1, 2.0)),
         transforms.ToTensor(),
         ]),
        'test': transforms.Compose(
        [transforms.Resize(224),
         transforms.CenterCrop(224),
         transforms.RandomHorizontalFlip(p=0.5),
         transforms.RandomRotation(degrees=(-10, 10)),
         transforms.RandomVerticalFlip(p=0.5),
         transforms.GaussianBlur(kernel_size= (5,5),sigma=(0.1, 2.0)),
         transforms.ToTensor(),
         ])}
    
    train_data = datasets.ImageFolder(root=data_path + '/train', transform=transform_dict['train'])
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)

    print('train_loader - len', len(train_loader))
    print('train_loader - type', type(train_loader))
    
    test_data = datasets.ImageFolder(root=data_path + '/test', transform=transform_dict['test'])
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)

    print('test_loader - len', len(test_loader))
    print('test_loader - type', type(test_loader))

    return train_loader, test_loader



In [ ]:
assert type(get_count_metrics('train')) is tuple
assert type(get_count_metrics('test')) is tuple
print(get_count_metrics('train'))
print(get_count_metrics('test'))

In [ ]:
train_loader, val_loader = load_data()

**Explore processed image**

In [ ]:
import torchvision
import matplotlib.pyplot as plt


def show_batch_images(dataloader):
    images, labels = next(iter(dataloader))
    grid = torchvision.utils.make_grid(images, padding=20)
    npgrid = grid.cpu().numpy()
    plt.figure(figsize=(30, 15))
    plt.imshow(np.transpose(npgrid, (1, 2, 0)), interpolation='nearest')
    print(labels)
    plt.title(label=["COVID19" if x==0 else "non-COVID19" for x in labels])
    plt.show()

 
for i in range(1):
    show_batch_images(train_loader)


**VGG16 Model Creation**

In [ ]:
num_classes = 3
vgg16 = models.vgg16_bn()
#vgg16.load_state_dict(torch.load("../input/vgg16bn/vgg16_bn.pth"))
print(vgg16.classifier[6].out_features)

for param in vgg16.features.parameters():
  param.requires_grad = False

num_features = vgg16.classifier[6].in_features
features = list(vgg16.classifier.children())[:-1] # Remove last layer
features.extend([nn.Linear(num_features, num_classes)]) # Add our layer with 4 outputs
vgg16.classifier = nn.Sequential(*features) # Replace the model classifier
print(vgg16)

In [ ]:
#Load vgg model and train on multiple epochs

n_epochs = 40
vgg16.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(vgg16.parameters(), lr= 1e-4) 

def train_model(model, train_dataloader, n_epoch=n_epochs, optimizer=optimizer, criterion=criterion):
    import torch.optim as optim

    model.train() # prep model for training
    _START_RUNTIME = time.time()
    
    for epoch in range(n_epoch):
        print(f"Epoch {epoch} starts")
        curr_epoch_loss = []
        for data, target in train_dataloader:
            if use_gpu:
                data, target = Variable(data.cuda()), Variable(target.cuda())
            
            outputs = model(data)
            loss = criterion(outputs, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            curr_epoch_loss.append(loss.cpu().data.numpy())
        print("Total train time = {:.2f} seconds".format(time.time() - _START_RUNTIME))    
        print(f"Epoch {epoch}: curr_epoch_loss={np.mean(curr_epoch_loss)}")
    return model

In [ ]:
import time


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")

In [ ]:
#print(model)
seed = 24
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
model = train_model(vgg16, train_loader)

In [ ]:
model.eval()
Y_pred = []
Y_test = []
predictions, actuals = list(), list()
for data, target in val_loader:
    if use_gpu:
      data, target = Variable(data.cuda()), Variable(target.cuda())

    Y_pred_orig = model(data)
    _, Y_pred_tag = torch.max(Y_pred_orig, dim = 1)
    Y_pred_tag = Y_pred_tag.cpu().detach().numpy()
    Y_pred_tag = Y_pred_tag.reshape(len(Y_pred_tag), 1)

    Y_test = target.cpu().numpy()
    Y_test = Y_test.reshape(len(Y_test), 1)

    predictions.append(Y_pred_tag)
    actuals.append(Y_test)


In [ ]:
Y_pred = np.concatenate(predictions, axis=0)
Y_test = np.concatenate(actuals, axis=0)

In [ ]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(Y_test, Y_pred)
print(("Validation Accuracy of Simple Vgg: " + str(acc)))